In [37]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)

from libs.common import *
from config.config import *

c:\Users\Admin\Data\ads_fraud_detection


# 3. Tiền xử lí dữ liệu

In [38]:
save_dir=f"{exps_dir}/exp2/exp_gau"

save_dir = f"{save_dir}"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [39]:
test_size=0.33
seed=42
train=pd.read_csv(f'{exps_dir}/exp2/train_EDA.csv')
test=pd.read_csv(f'{exps_dir}/exp2/test_EDA.csv')
train

,EVENT_ID,DISBURSED_AMOUNT,ASSET_COST,LTV,EMPLOYMENT_TYPE,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,...,ASSET_COST_new,ASSET_COST_bins,LTV_new,LTV_bins,PERFORM_CNS_SCORE_bins,PRI_NO_OF_ACCTS_new,PRI_NO_OF_ACCTS_bins,PRI_OVERDUE_ACCTS_new,PRI_OVERDUE_ACCTS_bins,age
0,462711,33484,62644,55.23,Salaried,1,1,0,0,0,...,62644.000000,Low,54340.127169,Low,High,54340.127169,More than One,54340.127169,NaN,12310
1,558674,66882,81187,84.37,Salaried,1,1,0,0,0,...,81187.000000,Extreme,54340.127169,Extreme,No History,54340.127169,One,54340.127169,NaN,11123
2,528251,59113,71757,84.87,Self employed,1,1,0,0,0,...,71757.000000,High,54340.127169,Extreme,High,54340.127169,More than One,54340.127169,NaN,11015
3,633950,56059,71307,81.34,Salaried,1,1,0,0,0,...,71307.000000,High,54340.127169,High,No History,54340.127169,One,54340.127169,NaN,19781
4,476747,56759,67100,85.69,Self employed,1,1,0,0,0,...,67100.000000,Medium,54340.127169,Extreme,No History,54340.127169,One,54340.127169,NaN,17984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186518,643379,55549,82230,70.78,Self employed,1,0,0,1,0,...,82230.000000,Extreme,54340.127169,Medium,No History,54340.127169,One,54340.127169,NaN,12111
186519,587710,60213,71857,84.89,Salaried,1,1,0,0,0,...,71857.000000,High,54340.127169,Extreme,No History,54340.127169,One,54340.127169,NaN,16128
186520,636081,52328,68757,78.54,Self employed,1,1,0,0,1,...,68757.000000,Medium,54340.127169,High,Low,54340.127169,More than One,54340.127169,One (or more),11200
186521,431351,77579,100360,79.71,Salaried,1,1,0,0,0,...,54340.127169,Extreme,54340.127169,High,Medium,54340.127169,More than One,54340.127169,One (or more),21460


In [40]:
train_con = train[['EMPLOYMENT_TYPE', 'MOBILENO_AVL_FLAG', 'AADHAR_FLAG', 'PAN_FLAG', 'VOTERID_FLAG',
                   'DRIVING_FLAG', 'PASSPORT_FLAG', 'PERFORM_CNS_SCORE', 'PERFORM_CNS_SCORE_DESCRIPTION', 
                   'PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE', 'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 
                   'SEC_OVERDUE_ACCTS', 'SEC_CURRENT_BALANCE', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT', 
                   'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 'CREDIT_HISTORY_LENGTH', 'NO_OF_INQUIRIES',
                   'LOAN_DEFAULT', 'DISBURSED_AMOUNT_new', 'ASSET_COST_new', 
                   'LTV_new', 'age']]

In [41]:
train_bin = train [['UNIQUEID', 'EMPLOYMENT_TYPE', 'MOBILENO_AVL_FLAG', 'AADHAR_FLAG', 'PAN_FLAG', 
                    'VOTERID_FLAG', 'DRIVING_FLAG', 'PASSPORT_FLAG', 'PERFORM_CNS_SCORE', 
                    'PERFORM_CNS_SCORE_DESCRIPTION', 'PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE',
                    'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 'SEC_OVERDUE_ACCTS', 'SEC_CURRENT_BALANCE',
                    'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT', 'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 
                    'CREDIT_HISTORY_LENGTH', 'NO_OF_INQUIRIES', 'LOAN_DEFAULT',
                    'DISBURSED_AMOUNT_bins', 'ASSET_COST_bins', 'LTV_bins',
                    'PERFORM_CNS_SCORE_bins', 'PRI_NO_OF_ACCTS_bins', 'PRI_OVERDUE_ACCTS_bins', 'age']]

In [42]:
test_con = test[['EMPLOYMENT_TYPE', 'MOBILENO_AVL_FLAG', 'AADHAR_FLAG', 'PAN_FLAG', 'VOTERID_FLAG',
                   'DRIVING_FLAG', 'PASSPORT_FLAG', 'PERFORM_CNS_SCORE', 'PERFORM_CNS_SCORE_DESCRIPTION', 
                   'PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE', 'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 
                   'SEC_OVERDUE_ACCTS', 'SEC_CURRENT_BALANCE', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT', 
                   'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 'CREDIT_HISTORY_LENGTH', 'NO_OF_INQUIRIES',
                   'LOAN_DEFAULT', 'DISBURSED_AMOUNT_new', 'ASSET_COST_new', 
                   'LTV_new', 'age']]

In [43]:
# Confusion Matrix

def plot_confusion_matrix(cm, classes,
                          normalize = False,
                          title = 'Confusion matrix"',
                          cmap = plt.cm.Blues) :
    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 0)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])) :
        plt.text(j, i, cm[i, j],
                 horizontalalignment = 'center',
                 color = 'white' if cm[i, j] > thresh else 'black')

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [44]:
scaler_data = StandardScaler() 
def scaleColumns(df, cols_to_scale):

    for col in cols_to_scale:

        df[col] = pd.DataFrame(scaler_data.fit_transform(pd.DataFrame(train_con[col])),columns=[col])

    return df

In [45]:
# scaled_df = scaleColumns(train_con,['PERFORM_CNS_SCORE','PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE', 
#                                  'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 'SEC_OVERDUE_ACCTS', 
#                                  'SEC_CURRENT_BALANCE', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT',
#                                  'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 'CREDIT_HISTORY_LENGTH', 
#                                  'NO_OF_INQUIRIES', 'DISBURSED_AMOUNT_new',
#                                  'ASSET_COST_new', 'LTV_new', 'age'])

# scaled_df.head()

In [46]:
# train_dummy = pd.get_dummies(scaled_df, prefix_sep='_', drop_first=True)
# train_dummy.head()

In [47]:
y_train = train_con[['LOAN_DEFAULT']]
x_train= train_con.loc[:, train_con.columns != 'LOAN_DEFAULT']
x_train.shape

(186523, 24)

In [48]:
x_train = x_train.fillna(0)
x_train.shape

(186523, 24)

In [49]:
y_train

,LOAN_DEFAULT
0,0
1,0
2,1
3,1
4,0
...,...
186518,0
186519,0
186520,0
186521,0


In [50]:
y_train.values

array([[0],
       [0],
       [1],
       ...,
       [0],
       [0],
       [0]], dtype=int64)

In [51]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train.LOAN_DEFAULT), y=y_train.LOAN_DEFAULT)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
np.savez(f'{save_dir}/class_weights_dict.npz', class_weights_dict = class_weights_dict)
class_weights_dict

{0: 0.6378163041991519, 1: 2.3140088827134457}

In [52]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_rosampled, y_rosampled = ros.fit_resample(train.drop(columns=['LOAN_DEFAULT']), train['LOAN_DEFAULT'])
X_rosampled = pd.DataFrame(X_rosampled)
y_rosampled = pd.DataFrame(y_rosampled)
oversampling_sample=pd.concat([X_rosampled, y_rosampled], axis=1)    

In [53]:
from sdv.metadata import SingleTableMetadata
metadata_sync = SingleTableMetadata()
metadata_sync.detect_from_dataframe(data=oversampling_sample)

In [54]:
from sdv.single_table import GaussianCopulaSynthesizer



gau_synthesizer = GaussianCopulaSynthesizer(metadata_sync,
                                            enforce_min_max_values=True,  
                                            enforce_rounding=True,
                                            default_distribution='gamma') 

# gau_synthesizer.fit(oversampling_sample)
# gau_synthesizer.save(
#     filepath=f'{save_dir}/gau_synthesizer.pkl'
# )

c:\Users\Admin\miniconda3\envs\gan_tabular\lib\site-packages\sdv\single_table\base.py:120: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.

c:\Users\Admin\miniconda3\envs\gan_tabular\lib\site-packages\sdv\single_table\base.py:105: UserWarning:

We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.



In [55]:
gau_synthesizer= GaussianCopulaSynthesizer.load(
    filepath=f'{save_dir}/gau_synthesizer.pkl'
)
print('GAU Synthesizer loaded')

GAU Synthesizer loaded


In [56]:
gau_synthetic_data = gau_synthesizer.sample(num_rows=len(train))
gau_synthetic_data.head()

,EVENT_ID,DISBURSED_AMOUNT,ASSET_COST,LTV,EMPLOYMENT_TYPE,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,...,ASSET_COST_bins,LTV_new,LTV_bins,PERFORM_CNS_SCORE_bins,PRI_NO_OF_ACCTS_new,PRI_NO_OF_ACCTS_bins,PRI_OVERDUE_ACCTS_new,PRI_OVERDUE_ACCTS_bins,age,LOAN_DEFAULT
0,549597,74255,37000,94.48,Self employed,1,1,0,0,0,...,Extreme,54340.127169,High,No History,54340.127169,One,54340.127169,NaN,13526,0
1,446685,59995,81676,75.79,Self employed,1,1,0,0,0,...,High,54340.127169,Extreme,No History,54340.127169,One,54340.127169,NaN,13538,0
2,626313,121822,369757,83.37,Self employed,1,1,1,0,0,...,Medium,54340.127169,Low,No History,54340.127169,One,54340.127169,NaN,15434,1
3,506809,51069,61794,64.48,Self employed,1,1,0,0,0,...,Medium,54340.127169,Low,No History,54340.127169,One,54340.127169,NaN,9646,0
4,518485,62077,217632,65.24,Self employed,1,0,0,0,0,...,Extreme,54340.127169,Low,Low,54340.127169,One,54340.127169,NaN,10368,0


In [57]:
from sdv.evaluation.single_table import run_diagnostic



diagnostic_gau = run_diagnostic(
    real_data=oversampling_sample,
    synthetic_data=gau_synthetic_data,
    metadata=metadata_sync
)




Generating report ...

(1/2) Evaluating Data Validity: |██████████| 48/48 [00:00<00:00, 80.71it/s]| 
Data Validity Score: 99.88%

(2/2) Evaluating Data Structure: |██████████| 1/1 [00:00<?, ?it/s]|
Data Structure Score: 100.0%

Overall Score (Average): 99.94%



In [58]:
# from sdv.evaluation.single_table import evaluate_quality



# diagnostic_gau = evaluate_quality(
#     real_data=oversampling_sample,
#     synthetic_data=gau_synthetic_data,
#     metadata=metadata_sync
# )




In [59]:
from sklearn.utils import shuffle
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)

## Gau data
gau_data=gau_synthetic_data[gau_synthetic_data.LOAN_DEFAULT==1]
gau_sample=pd.concat([train, gau_data] , axis=0, ignore_index=True)
X_gausampled, y_gausampled = rus.fit_resample(gau_sample.drop(columns=['LOAN_DEFAULT']), gau_sample['LOAN_DEFAULT'])
X_gausampled = pd.DataFrame(X_gausampled)
y_gausampled = pd.DataFrame(y_gausampled)
gau_sample=pd.concat([X_gausampled, y_gausampled], axis=1)    
gau_sample = shuffle(gau_sample, random_state=42)
print(gau_sample.LOAN_DEFAULT.value_counts())

LOAN_DEFAULT
0    133443
1    133443
Name: count, dtype: int64


In [60]:
gau_sample

,EVENT_ID,DISBURSED_AMOUNT,ASSET_COST,LTV,EMPLOYMENT_TYPE,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,...,ASSET_COST_bins,LTV_new,LTV_bins,PERFORM_CNS_SCORE_bins,PRI_NO_OF_ACCTS_new,PRI_NO_OF_ACCTS_bins,PRI_OVERDUE_ACCTS_new,PRI_OVERDUE_ACCTS_bins,age,LOAN_DEFAULT
156888,513161,43133,69399,67.41,Self employed,1,0,0,1,0,...,Medium,54340.127169,Low,High,54340.127169,More than One,54340.127169,NaN,21452,0
186804,535248,40568,123243,55.25,Self employed,1,1,0,0,0,...,High,54340.127169,Extreme,High,54340.127169,More than One,54340.127169,NaN,14807,1
274645,506402,51891,37000,81.98,Self employed,1,1,0,0,0,...,High,54340.127169,High,No History,54340.127169,More than One,54340.127169,NaN,14804,1
58720,419121,72817,85298,86.75,Salaried,1,1,0,0,0,...,Extreme,54340.127169,Extreme,No History,54340.127169,One,54340.127169,NaN,14667,1
215647,588383,68962,246455,66.36,Self employed,1,1,0,0,0,...,Low,54340.127169,Medium,No History,54340.127169,One,54340.127169,NaN,12259,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100533,418216,67600,91500,74.97,Salaried,1,1,0,0,0,...,Extreme,54340.127169,Medium,Very Low,54340.127169,One,54340.127169,NaN,10338,0
271955,439531,45264,37000,78.36,Salaried,1,1,0,0,0,...,Extreme,54340.127169,Low,No History,54340.127169,One,54340.127169,NaN,13139,1
89060,489098,72817,100300,74.78,Salaried,1,1,0,0,0,...,Extreme,54340.127169,Medium,High,54340.127169,More than One,54340.127169,NaN,19264,0
61855,508395,79033,112910,71.74,Self employed,1,0,0,0,1,...,Extreme,54340.127169,Medium,No History,54340.127169,One,54340.127169,NaN,22307,1


In [61]:
label_encoders = {}
for column in gau_sample.select_dtypes(include=['object']).columns:
    label_encoder = LabelEncoder()
    gau_sample[column] = label_encoder.fit_transform(gau_sample[column])
    label_encoders[column] = label_encoder  # Lưu trữ label encoder nếu cần sau này

gau_sample.head()

,EVENT_ID,DISBURSED_AMOUNT,ASSET_COST,LTV,EMPLOYMENT_TYPE,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,...,ASSET_COST_bins,LTV_new,LTV_bins,PERFORM_CNS_SCORE_bins,PRI_NO_OF_ACCTS_new,PRI_NO_OF_ACCTS_bins,PRI_OVERDUE_ACCTS_new,PRI_OVERDUE_ACCTS_bins,age,LOAN_DEFAULT
156888,513161,43133,69399,67.41,2,1,0,0,1,0,...,3,54340.127169,2,0,54340.127169,0,54340.127169,1,21452,0
186804,535248,40568,123243,55.25,2,1,1,0,0,0,...,1,54340.127169,0,0,54340.127169,0,54340.127169,1,14807,1
274645,506402,51891,37000,81.98,2,1,1,0,0,0,...,1,54340.127169,1,3,54340.127169,0,54340.127169,1,14804,1
58720,419121,72817,85298,86.75,1,1,1,0,0,0,...,0,54340.127169,0,3,54340.127169,1,54340.127169,1,14667,1
215647,588383,68962,246455,66.36,2,1,1,0,0,0,...,2,54340.127169,3,3,54340.127169,1,54340.127169,1,12259,1


In [62]:
# gau_sample = pd.get_dummies(gau_sample, prefix_sep='_', drop_first=True)
# gau_sample.head()

In [ ]:
gau_sample = scaleColumns(gau_sample,['PERFORM_CNS_SCORE','PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE', 
                                 'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 'SEC_OVERDUE_ACCTS', 
                                 'SEC_CURRENT_BALANCE', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT',
                                 'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 'CREDIT_HISTORY_LENGTH', 
                                 'NO_OF_INQUIRIES', 'DISBURSED_AMOUNT_new',
                                 'ASSET_COST_new', 'LTV_new', 'age'])

gau_sample.head()

,EMPLOYMENT_TYPE,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,PASSPORT_FLAG,PERFORM_CNS_SCORE,PERFORM_CNS_SCORE_DESCRIPTION,PRI_ACTIVE_ACCTS,...,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,CREDIT_HISTORY_LENGTH,NO_OF_INQUIRIES,LOAN_DEFAULT,DISBURSED_AMOUNT_new,ASSET_COST_new,LTV_new,age
0,Salaried,1,1,0,0,0,0,1.340292,C-Very Low Risk,2.031893,...,-0.025404,-0.020499,-0.252023,-0.009193,1.127407,0,-1.810134,-0.561253,2.910383e-11,-0.734524
1,Salaried,1,1,0,0,0,0,-0.855060,No Bureau History Available,-0.533268,...,-0.087314,-0.020499,-0.252023,-0.568043,-0.291316,0,1.178058,1.368304,2.910383e-11,-1.065269
2,Self employed,1,1,0,0,0,0,1.325519,C-Very Low Risk,1.005829,...,-0.058623,-0.020499,-0.252023,-0.428330,-0.291316,1,0.482948,0.387032,2.910383e-11,-1.095362
3,Salaried,1,1,0,0,0,0,-0.855060,No Bureau History Available,-0.533268,...,-0.087314,-0.020499,-0.252023,-0.568043,-0.291316,1,0.209700,0.340206,2.910383e-11,1.347191
4,Self employed,1,1,0,0,0,0,-0.855060,No Bureau History Available,-0.533268,...,-0.087314,-0.020499,-0.252023,-0.568043,-0.291316,0,0.272331,-0.097568,2.910383e-11,0.846476


In [64]:
x_train = gau_sample.drop(columns=['LOAN_DEFAULT'])
y_train = gau_sample['LOAN_DEFAULT']

In [65]:
from sklearn.model_selection import train_test_split
x_train.to_csv(f'{save_dir}/x_train.csv', index=False)
y_train.to_csv(f'{save_dir}/y_train.csv', index=False)


In [66]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
print(f"+ X_train: {len(x_train)}")
for fold, (train_idx, valid_idx) in enumerate(kfold.split(x_train, y_train)):
    print(f'Fold {fold}: ')
    print(f'+ train_idx: {train_idx}')
    print(f'+ valid_idx: {valid_idx}')
    print(f'+ train / valid: {valid_idx}')
    pass

+ X_train: 266886
Fold 0: 
+ train_idx: [     0      1      2 ... 266881 266883 266884]
+ valid_idx: [     6     11     12 ... 266875 266882 266885]
+ train / valid: [     6     11     12 ... 266875 266882 266885]
Fold 1: 
+ train_idx: [     2      3      5 ... 266882 266884 266885]
+ valid_idx: [     0      1      4 ... 266877 266878 266883]
+ train / valid: [     0      1      4 ... 266877 266878 266883]
Fold 2: 
+ train_idx: [     0      1      2 ... 266882 266883 266885]
+ valid_idx: [     3     21     23 ... 266873 266880 266884]
+ train / valid: [     3     21     23 ... 266873 266880 266884]
Fold 3: 
+ train_idx: [     0      1      3 ... 266883 266884 266885]
+ valid_idx: [     2      7      8 ... 266859 266871 266872]
+ train / valid: [     2      7      8 ... 266859 266871 266872]
Fold 4: 
+ train_idx: [     0      1      2 ... 266883 266884 266885]
+ valid_idx: [     5      9     10 ... 266876 266879 266881]
+ train / valid: [     5      9     10 ... 266876 266879 266881]


In [67]:
# for column in outliers_list:
#     data_test = remove_outliers_iqr(data_test, column)
# data_test['collision_type'].replace('?',np.nan,inplace=True)
# data_test['collision_type'].fillna(method='ffill',inplace=True)
# data_test[['property_damage','police_report_available']].replace('?','Unknow',inplace=True)
# (data_test=='?').sum()
scaled_df = scaleColumns(test_con,['PERFORM_CNS_SCORE','PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE', 
                                 'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 'SEC_OVERDUE_ACCTS', 
                                 'SEC_CURRENT_BALANCE', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT',
                                 'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 'CREDIT_HISTORY_LENGTH', 
                                 'NO_OF_INQUIRIES', 'DISBURSED_AMOUNT_new',
                                 'ASSET_COST_new', 'LTV_new', 'age'])

test_dummy = pd.get_dummies(scaled_df, prefix_sep='_', drop_first=True)
y_test = test_dummy[['LOAN_DEFAULT']]
x_test= test_dummy.loc[:, test_dummy.columns != 'LOAN_DEFAULT']
x_test.to_csv(f'{save_dir}/x_test.csv', index=False)
y_test.to_csv(f'{save_dir}/y_test.csv', index=False)